In [0]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os 

In [2]:
torch.__version__

'1.4.0'

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls

drive  sample_data


In [0]:
#with open('/gdrive/My Drive/foo.txt', 'w') as f:
 # f.write('Hello Google Drive!')
#!cat '/gdrive/My Drive/foo.txt'

In [0]:
#%cd drive/My Drive/AlphaPose/AlphaPose-pytorch
#input_path = "/content/drive/My Drive/AlphaPose/AlphaPose-pytorch/fi_data"


In [0]:
#!python3 datalabeler_111.py

In [7]:
%cd drive/My Drive/Resnet_Beta-Version
!ls

/content/drive/My Drive/Resnet_Beta-Version
 alpha-dataset
'alpha-Resnet(Raw+Face+Hand).ipynb'
'alpha-Resnet(Raw+Face).ipynb'
'alpha-Resnet(Raw+Hand).ipynb'
'alpha-Resnet(Raw).ipynb'
'Arrange Dataset.ipynb'
 Arslan_label_falling
'beta-Resnet(Raw+Face+Hand).ipynb'
'beta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'beta-Resnet(Raw+Face).ipynb'
'beta-Resnet(Raw+Hand).ipynb'
'beta-Resnet(Raw).ipynb'
 colored_dataset_Arslan
 coloured_dataset_Govi
'cropped_Resnet(Raw).ipynb'
 epoch_65.pth
 extract_faceandhand_jorehgaye.ipynb
 fallingDemo.ipynb
 fallingDemo_modified_2.py
 fallingDemo_modified.py
 fallingDemo.py
 Found_missing_detections
 frames
 GrayScale_dataset_Arslan
 GrayScale_dataset_Govi
 loss
 MissingDetections_Arslan_colored
 MissingDetections_Govi_coloured
 out.jpg
 putTXT_rightplace.ipynb
'Testbeta-Resnet(Raw+Face+Hand).ipynb'
'Testbeta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'testbeta-Resnet(Raw+Face).ipynb'
'Testbeta-Resnet(Raw+Hand).ipynb'
'Testbeta-Resnet(Raw).ipynb'

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

In [0]:
class LimitDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, n):
        self.dataset = dataset
        self.n = n

    def __len__(self):
        return self.n


    def __getitem__(self, i):
        return self.dataset[i]

In [12]:

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform
        




    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

       
        img_path = path
       
        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
       
        #Croping images to feeed 
        if (os.path.isfile(filename.split('.')[0] + '-bbox.txt')):
          cpath  = img_path.replace(filename, filename.split('.')[0] + '-bbox.txt')
          image  = self.crop_text(cpath,image)
    

        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
        
        return image, target


    def crop_text(self,path,image):
        try:
              fr = open(path)
              # print ("fr",cat+"/yolo/" + imagePath.split("/")[-1].split('.')[0]+".txt")

              line = fr.readline().split(" ")

              width, height = image.size 
        
              # Setting the points ffloor cropped image
              b_w = float(line[3]) * width
              b_h = float(line[4]) * height

              t_left = (float(line[1]) * width) - (b_w/2)
              t_right = (float(line[2]) * height) - (b_h/2)

              b_left = (float(line[1]) * width) + (b_w/2)
              b_right = (float(line[2]) * height) + (b_h/2)

              # print("w",width," h",height)

              # print (t_left)
              # print (t_right)
              # print (b_left)
              # print (b_right)

              image = image.crop((t_left, t_right, b_left, b_right))
              
        except FileNotFoundError:
              pass
        return image 
   
"""
Test Code


path, target = self.samples[index]
sample = self.loader(path)
if self.transform is not None:
  sample = self.transform(sample)
if self.target_transform is not None:
  target = self.target_transform(target)
# this is where you load your resnet data
resnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth
resnet = torch.load(resnet_path)  # load the stored features
return sample, resnet, target
"""
#dataset = MyImageFolder("falling_dataset/val/")
#re, labels = dataset[1:3]
#print(re)
#print(labels)
#re[2].save('pred_face.jpeg')
#re[3].save('pred_hand.jpeg')

"\nTest Code\n\n\npath, target = self.samples[index]\nsample = self.loader(path)\nif self.transform is not None:\n  sample = self.transform(sample)\nif self.target_transform is not None:\n  target = self.target_transform(target)\n# this is where you load your resnet data\nresnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth\nresnet = torch.load(resnet_path)  # load the stored features\nreturn sample, resnet, target\n"

In [0]:
#%cd 

In [16]:
#input_path_train = '/content/drive/My Drive/AlphaPose/AlphaPose-pytorch/data/test111/Cleaned_dataset'
input_path = './alpha-dataset/'
image_datasets = {
    'train': 
    MyImageFolder(input_path+"train_m" , data_transforms['train']),
    'validation': 
    MyImageFolder(input_path+"val_m" , data_transforms['validation'])
}

print(image_datasets['train'][0])

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=8),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=16,
                                shuffle=True,
                                num_workers=8)  # for Kaggle
}

(tensor([[[ 1.3070,  1.3242,  1.3584,  ...,  0.5022,  0.4851,  0.4851],
         [ 1.3070,  1.3242,  1.3584,  ...,  0.5022,  0.4851,  0.4851],
         [ 1.2899,  1.2899,  1.3242,  ...,  0.5022,  0.4851,  0.4679],
         ...,
         [-1.7069, -1.7412, -1.7069,  ...,  0.0056, -0.0458, -0.0458],
         [-1.7412, -1.7583, -1.7240,  ..., -0.0287, -0.0801, -0.1143],
         [-1.7412, -1.7583, -1.7240,  ..., -0.0458, -0.0972, -0.1486]],

        [[ 1.4657,  1.4832,  1.5007,  ...,  0.5553,  0.5553,  0.5553],
         [ 1.4657,  1.4832,  1.5007,  ...,  0.5553,  0.5553,  0.5553],
         [ 1.4482,  1.4482,  1.4832,  ...,  0.5553,  0.5553,  0.5378],
         ...,
         [-1.5280, -1.5630, -1.5455,  ...,  0.0301,  0.0301,  0.0126],
         [-1.5455, -1.5805, -1.5630,  ...,  0.0126, -0.0224, -0.0574],
         [-1.5455, -1.5805, -1.5630,  ..., -0.0224, -0.0399, -0.0924]],

        [[ 1.6814,  1.6988,  1.7337,  ...,  0.6182,  0.6182,  0.6182],
         [ 1.6814,  1.6988,  1.7337,  ...,  

In [17]:
#image_datasets['train']=LimitDataset(image_datasets['train'], n=1500)
dataloaders['train'].dataset.classes


['Falling', 'NonFalling']

In [18]:
dataloaders['validation'].dataset.classes

['Falling', 'NonFalling']

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [20]:
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.00001)

In [0]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']: #, 'validation'
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 0
            with torch.set_grad_enabled(phase == 'train'):
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    outputs = model(inputs)
                    #print ("Output " +str(outputs.size()))
                    #print ("Labels"+str(labels.size()))
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    _, preds = torch.max(outputs, 1)
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    count = count+1                
                    if (count%40==0):
                      print (count)
                
                

                epoch_loss = running_loss / len(image_datasets[phase])
                epoch_acc = running_corrects.double() / len(image_datasets[phase])

                print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
        torch.save(model, '/content/drive/My Drive/Resnet/clone_weights/raw_2/Epoch_%d.pth'%(epoch))
    return model

In [0]:
#count = 1
#torch.save(model, 'weights/pytorch/fallweightsresNet_3_12_2019%d.pth'%(count))

In [25]:
model_trained = train_model(model, criterion, optimizer, num_epochs=100)


Epoch 1/100
----------
40
train loss: 0.6452, acc: 0.7458
40
validation loss: 0.6274, acc: 0.7834
Epoch 2/100
----------
40
train loss: 0.6170, acc: 0.7831
40
validation loss: 0.5990, acc: 0.7969
Epoch 3/100
----------
40
train loss: 0.5893, acc: 0.7975
40
validation loss: 0.5730, acc: 0.8150
Epoch 4/100
----------
40
train loss: 0.5641, acc: 0.8078
40
validation loss: 0.5491, acc: 0.8096
Epoch 5/100
----------
40
train loss: 0.5408, acc: 0.8137
40
validation loss: 0.5271, acc: 0.8276
Epoch 6/100
----------
40
train loss: 0.5222, acc: 0.8199
40
validation loss: 0.5097, acc: 0.8249
Epoch 7/100
----------
40
train loss: 0.5023, acc: 0.8260
40
validation loss: 0.4918, acc: 0.8412
Epoch 8/100
----------
40
train loss: 0.4882, acc: 0.8286
40
validation loss: 0.4775, acc: 0.8412
Epoch 9/100
----------
40
train loss: 0.4727, acc: 0.8312
40
validation loss: 0.4649, acc: 0.8403
Epoch 10/100
----------
40
train loss: 0.4604, acc: 0.8335
40
validation loss: 0.4537, acc: 0.8484
Epoch 11/100
------

In [0]:
!mkdir weights
!mkdir weights/pytorch

In [0]:
torch.save(model_trained.state_dict(), 'weights/pytorch/weightsE3.h5')

In [0]:
# model = models.resnet50(pretrained=False).to(device)
# model.fc = nn.Sequential(
#                nn.Linear(2048, 128),
#                nn.ReLU(inplace=True),
#                nn.Linear(128, 2)).to(device)
model = (torch.load('weights/pytorch/fallweightsresNet_3_12_201990.pth'))

In [0]:
!ls

darknet		      examples	      opt.py		train_sppe
data		      fallmodel.pth   output		video_ann.py
datalabeler_101.py    fi_data	      PoseFlow		video_demo.py
datalabeler_111.py    fn.py	      pPose_nms.py	webcam_demo.py
datalabeler_249.py    img.png	      __pycache__	weights
dataloader.py	      LICENSE	      README.md		yolo
dataloader_webcam.py  matching.py     requirements.txt
demo.py		      models	      resnetWeights
doc		      online_demo.py  SPPE


In [0]:
import glob
import random
pathFalling = './fi_data/train/Falling/'
pathNonFalling = './fi_data/train/NonFalling/'
accurateFalling = 0
accurateNonFalling = 0

list1= [f for f in glob.glob(pathFalling + "*.jpg", recursive=True)]
list2= [f for f in glob.glob(pathNonFalling + "*.jpg", recursive=True)]
validation_img_pathsFalling=random.sample(list1,100)
validation_img_pathsNonFalling = random.sample(list2,100)
img_listFalling = [Image.open('./' + img_path) for img_path in validation_img_pathsFalling]
img_listNonFalling = [Image.open('./' + img_path) for img_path in validation_img_pathsNonFalling]
print (len(img_listFalling))
print (len(img_listNonFalling))

100
100


In [0]:
validation_batchFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listFalling])
validation_batchNonFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listNonFalling])

In [0]:
pred_logits_tensorFalling = model(validation_batchFalling)
pred_logits_tensorFalling
pred_logits_tensorNonFalling = model(validation_batchNonFalling)
pred_logits_tensorNonFalling

tensor([[-1.5283e+00,  9.0661e-01],
        [-2.2185e+00,  2.1231e+00],
        [-2.7703e+00,  3.2524e+00],
        [-2.4615e+00,  2.0275e+00],
        [-1.1705e+00,  9.6872e-01],
        [-4.7061e-01,  8.8000e-01],
        [-1.1260e+00,  6.5979e-01],
        [-3.8996e+00,  4.1767e+00],
        [-5.8298e-01,  2.0960e-01],
        [-7.5061e-01,  5.5574e-01],
        [-2.2707e+00,  2.1771e+00],
        [-1.5751e+00,  1.3934e+00],
        [-9.6311e-01,  3.2051e-01],
        [-2.7319e+00,  2.2038e+00],
        [-9.1111e-02, -3.2099e-03],
        [ 7.4642e-02, -3.1217e-01],
        [-1.6257e+00,  1.6039e+00],
        [-3.4561e+00,  3.4454e+00],
        [-3.1866e+00,  2.6508e+00],
        [-6.3874e-01,  3.6337e-01],
        [-1.6754e+00,  1.5005e+00],
        [-2.6561e+00,  2.5272e+00],
        [-4.2902e+00,  3.4453e+00],
        [-5.9180e+00,  5.5812e+00],
        [-1.1807e+00,  3.4439e-01],
        [ 5.9854e-01, -1.1552e+00],
        [-1.0886e+00,  1.4284e+00],
        [-1.3321e+00,  6.760

In [0]:
pred_probsFalling = F.softmax(pred_logits_tensorFalling, dim=1).cpu().data.numpy()
print (pred_probsFalling)
pred_probsNonFalling = F.softmax(pred_logits_tensorNonFalling, dim=1).cpu().data.numpy()
print (pred_probsNonFalling)

[[0.44844374 0.5515562 ]
 [0.12725133 0.8727487 ]
 [0.09804448 0.9019555 ]
 [0.00425655 0.99574345]
 [0.8194356  0.18056442]
 [0.24643923 0.7535607 ]
 [0.13835269 0.8616473 ]
 [0.1574074  0.84259254]
 [0.8485861  0.1514139 ]
 [0.32346088 0.6765391 ]
 [0.41357878 0.58642125]
 [0.15117285 0.8488271 ]
 [0.6003375  0.39966255]
 [0.02290936 0.9770906 ]
 [0.10023551 0.89976454]
 [0.29872367 0.70127636]
 [0.04371396 0.956286  ]
 [0.61934406 0.38065597]
 [0.30597326 0.6940267 ]
 [0.16753031 0.8324697 ]
 [0.66433    0.33567002]
 [0.84191    0.15808995]
 [0.6628018  0.33719823]
 [0.4977156  0.50228435]
 [0.87176895 0.12823105]
 [0.02149463 0.9785054 ]
 [0.48655316 0.5134468 ]
 [0.44041517 0.55958486]
 [0.1114507  0.88854927]
 [0.5616096  0.4383904 ]
 [0.570884   0.42911607]
 [0.32838956 0.6716105 ]
 [0.42478982 0.5752102 ]
 [0.5371839  0.46281612]
 [0.02694354 0.9730565 ]
 [0.61242115 0.3875789 ]
 [0.21000993 0.7899901 ]
 [0.32981074 0.6701893 ]
 [0.23768397 0.76231605]
 [0.846486   0.15351407]


In [0]:
i=0
accurateFalling=0
for i in range(len(pred_probsFalling)):
  if (pred_probsFalling[i][0]> 0.5):
    accurateFalling= accurateFalling +1 
i=0
accurateNonFalling=0
for i in range(len(pred_probsNonFalling)):
  if (pred_probsNonFalling[i][1]> 0.5):
    accurateNonFalling= accurateNonFalling +1 


In [0]:
overallAccuracy = (accurateNonFalling+accurateFalling)/(len(img_listFalling)+len(img_listNonFalling))

In [0]:
print (accurateFalling)
print (accurateNonFalling)

41
92


In [0]:
print (overallAccuracy)

0.665


In [0]:
ig, axs = plt.subplots(1, len(img_listFalling), figsize=(150, 150))
for i, img in enumerate(img_listFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsFalling[i,0],
                                                          100*pred_probsFalling[i,1]))
    ax.imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
ig, axs = plt.subplots(1, len(img_listNonFalling), figsize=(150, 150))
for i, img in enumerate(img_listNonFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsNonFalling[i,0],
                                                          100*pred_probsNonFalling[i,1]))
    ax.imshow(img)

Output hidden; open in https://colab.research.google.com to view.